In [21]:
import random
from datetime import datetime
import pandas as pd

In [24]:
def random_roll():
    last_time = datetime.now().timestamp()
    while True:
        time_now = datetime.now().timestamp()
        if last_time != time_now:
            random.seed(time_now)
            last_time = time_now
        else:
            random.seed(random.random())
        yield random.random()
rnd = random_roll()

class Model:
    def __init__(self, config):
        self.config = config # вхідні параметри зберігаємо
        self.store = []
        self.attributes = ["day","start_stock","number","demand","ordered","replenished","end_stock"]
        self.deficit_days = 0
        self.day = 0
        self.end_stock = config.start_stock # треба пам'ятати, який запас був учора ввечорі
        self.order_days = 0 # лічільник того, скільки днів щоби до запасу надійшло поповнення після замовлення
        # перетворимо зважені ймовірності на коефіцієнти (30 -> 0.3)
        probabilities = []
        w_sum = sum(config.weights)
        for w in config.weights:
            probabilities.append(w/w_sum)
        # додамо акумульовані ймовірності, що вони дорівнюють сумі усіх попередніх ймовірностей
        p_accumulator = 0
        self.accumulation = []
        for p in probabilities:
            p_accumulator += p
            self.accumulation.append(p_accumulator)
    def time_increment(self):
        # дії на початку дня
        self.day += 1
        start_stock = self.end_stock
        replenished = 0
        ordered = 0
        # Надходження замовлення
        if self.order_days > 0:
            self.order_days -= 1
            if self.order_days == 0:
                replenished = self.config.replenish_quantity
                start_stock += self.config.replenish_quantity
        else: # Замовлення немає
            # треба розмістити замовлення, якщо запас просів нижче порогу 
            if start_stock <= self.config.replenish_condition:
                self.order_days = self.config.cycle_days
                ordered = self.config.replenish_quantity
        number = next(rnd)
        demand = 0
        index = 0
        for n in self.accumulation:
            if n <= number:
                demand = self.config.quantities[index]
            index += 1
        self.end_stock = start_stock - demand
        if self.end_stock < 0:
            self.deficit_days += 1
        return [
            self.day,
            start_stock,
            number,
            demand,
            ordered,
            replenished,
            self.end_stock
        ]
    def model(self, timespan = 10):
        for _ in range(timespan):
            self.store.append(self.time_increment())
        df = pd.DataFrame(self.store, columns=self.attributes)
        print(df)
        return(df)

class Object:
    def __init__(self, **attributes):
        self.__dict__.update(attributes)

In [25]:
demand_config = Object(
    start_stock = 10,
    weights = [10, 30, 30, 20, 5, 5],
    quantities = [1, 2, 3, 4, 5, 6],
    cycle_days = 2,
    replenish_quantity = 8,
    replenish_condition = 6
)

timespan = 10000
model = Model(demand_config)
df = model.model(timespan)

        day  start_stock    number  demand  ordered  replenished  end_stock
0         1           10  0.263194       1        0            0          9
1         2            9  0.782346       3        0            0          6
2         3            6  0.836466       3        8            0          3
3         4            3  0.228903       1        0            0          2
4         5           10  0.605386       2        0            8          8
...     ...          ...       ...     ...      ...          ...        ...
9995   9996            7  0.273253       1        0            0          6
9996   9997            6  0.461227       2        8            0          4
9997   9998            4  0.470500       2        0            0          2
9998   9999           10  0.359141       1        0            8          9
9999  10000            9  0.090465       0        0            0          9

[10000 rows x 7 columns]


In [20]:
print(f"{model.deficit_days} deficit days out of {timespan} days modeled")
print(f"with {model.deficit_days/timespan*100}% of deficit probability")

667 deficit days out of 10000 days modeled
with 6.67% of deficit probability
